# Homework 3
#### Author: Gustavo Soares

##### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import bql
bq = bql.Service()

## Introduction

Bloomberg calculates a number of indics for sectors of the US REITs market. They are capitalization-weighted indices of Real Estate Investment Trusts having a market capitalization of USD 15mn or greater. The indices are rebalanced semi-annually in February and August.

Let's grab the total return indices for some subsectors of the US REITs market:

In [14]:
BBG_INDICES_DICT = {
    'Industrial' : 'BBREINDW Index',
    'Diversified' : 'BBREDIVR Index',
    'Self storage' : 'BBREPBST Index',
    'Single tenant' : 'BBREOUTL Index',
    'Healthcare' : 'BBREHLTH Index',
    'Apartment' : 'BBREAPT Index',
    'Office' : 'BBREOFPY Index',
    'Shopping' : 'BBRESHOP Index',
    'Hotel' : 'BBREHOTL Index',
    'Regional Malls' : 'BBREMALL Index',
}

request = "get(TOTAL_RETURN_INDEX(DATES=RANGE('-10Y', '0D'))) for(["
for x in BBG_INDICES_DICT.values():
    request += "'" + x + "',"
request = request[:-1] + "])"
res = bq.execute(request)
usreits = bql.combined_df(res)
usreits.to_csv('usreits.csv')
usreits.head()

,DATE,"TOTAL_RETURN_INDEX(DATES=RANGE('-10Y','0D'))"
ID,,
BBREINDW Index,2011-05-11,1.000000
BBREINDW Index,2011-05-12,1.006306
BBREINDW Index,2011-05-13,0.989427
BBREINDW Index,2011-05-14,0.989427
BBREINDW Index,2011-05-15,0.989427


## Q1: PCA

Using what you know about [PCA](https://github.com/Finance-Hub/FinanceHubMaterials/blob/master/Quantitative%20Finance%20Lectures/lecture03/pca/PCA.ipynb), how many factors are needed to explain the US REITs market? If you are feeling a bit adventurous, you may want to try to check how the different sectors [cluster](https://github.com/Finance-Hub/FinanceHubMaterials/blob/master/Quantitative%20Finance%20Lectures/lecture03/pca/fx_pca_case.ipynb) together.

## Q2: Factors

Let's assume that the US REITS market is well explained by two factors. Let's grab the S&P 500 excess returns and the US 10Y Treasuries excess returns as the two market factors. Let's also grab the 3M Libor in order to calculate the excess returns for the Bloomberg REITs indices above (The SPXFP Index and the SPUSTTP Index are futures rolling indices, so they are already in excess returns):

In [16]:
FACTOR_DICT = {
    'SP500' : 'SPXFP Index',
    'USTs' : 'SPUSTTP Index',
    'Libor' : 'US0003M Index'
}

tickers = list(FACTOR_DICT.values())
fields = {'price' : bq.data.px_last(fill='PREV', start=etf_df['DATE'].min(), end=etf_df['DATE'].max())}
req = bql.Request(tickers, fields)
res = bq.execute(req)
rates_df = bql.combined_df(res)
rates_df.to_csv('factors.csv')
rates_df.head()

,DATE,CURRENCY,price
ID,,,
SPXFP Index,2011-05-11,USD,111.48
SPXFP Index,2011-05-12,USD,112.21
SPXFP Index,2011-05-13,USD,111.09
SPXFP Index,2011-05-14,USD,111.09
SPXFP Index,2011-05-15,USD,111.09


Now, using what you learned about [Linear Beta Pricing Models](http://localhost:8888/notebooks/FinanceHubMaterials/Quantitative%20Finance%20Lectures/lecture03/linear_models.ipynb), test if these two factors can explain the return in the set of test portfolios composed by the Bloomberg indices for the different subsectors of the US REITs market. What do you think? Is real estate just a composite asset? It's basically a combination of equity risk premium and bond risk premium? Or is there a statistically meaningful difference in this market?

## Q3 Fama-French Factors

Try to answer exactly the same question as in Q2 but instead use the Fama-French factors found at [on Kenneth French's web page](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html). You may use ```pandas-datareader``` to download the monthly returns data. Note that you will need to match the dates before running the regression.